In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import re

import ipywidgets as widgets
from ipywidgets import GridBox, Button
from IPython.display import display, clear_output

import os
from datetime import datetime

### Read progress and records files

In [20]:
environments = {
    "water_tank": {
        "base_dir": "../logs/ppo/double_water_tank/trainings",
        "tracked_point": "x2",
    },
    "CPAP": {
        "base_dir": "../logs/ppo/CPAP",
        "tracked_point": "x3",
    },
}


env = environments["water_tank"]
base_dir = env["base_dir"]
tracked_point = env["tracked_point"]

index = None # -1
folder_names = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))][:index]
def folder_to_datetime(folder_name):
    """
    Função para converter o nome da pasta em um objeto datetime.
    Combinada com max(), é possível ordenar de forma decrescente pastas por data da mais recente (max) para antiga (min).
    O nome da pasta DEVE estar no formato '%d-%m-%H%M' ou ter uma substring nesse formato para funcionar corretamente.
    """
    pattern = r'\d{2}-\d{2}-\d{4}'
    match = re.search(pattern, folder_name)
    if match:
        date_str = match.group(0)
        date_str = datetime.strptime(date_str, '%d-%m-%H%M')
        return (date_str.month, date_str.day)
    else:
        raise ValueError(f"Formato de data inválido na pasta: {folder_name}")


folder_names.sort(key=folder_to_datetime)

latest_folder = folder_names[-1] # max(folder_names, key=folder_to_datetime)
selected_folder = latest_folder
selected_index = len(folder_names) - 1


# print(folder_names)
# print(f"A pasta mais recente é: {latest_folder}")

In [21]:
def are_required_files_present(folder: str):
    return all([
        os.path.exists(f"{base_dir}/{folder}/progress.csv"),
        os.path.exists(f"{base_dir}/{folder}/records.csv"),
        # os.path.exists(f"{base_dir}/{folder}/only_pid_records.csv"),
    ])

In [22]:
def load_dfs(folder: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    if not are_required_files_present(folder):
        raise FileNotFoundError("Arquivos necessários não encontrados.")
    
    progress_df = pd.read_csv(f"{base_dir}/{folder}/progress.csv")
    rec_df = pd.read_csv(f"{base_dir}/{folder}/records.csv")

    # Find the index of the last episode start
    mask = rec_df["steps_in_episode"] == 1
    start_episode_indexes = mask[mask].index.to_list()
    last_episode_start_idx = start_episode_indexes[-1]

    # Filter the dataframe for the last episode
    last_episode_df = rec_df.loc[last_episode_start_idx:]

    return progress_df, rec_df, last_episode_df

progress_df, rec_df, last_episode_df = load_dfs(latest_folder)

In [23]:
progress_df.head()

,train/policy_gradient_loss,train/approx_kl,train/clip_range,train/iteration,train/learning_rate,train/entropy_loss,train/explained_variance,train/clip_fraction,train/std,train/n_updates,train/value_loss,train/loss
0,0.0,0.000004,0.2,8,0.0003,-7.505897,NaN,0.0,440.807495,20000,2.227601e-08,-0.150145


In [24]:
rec_df

,x1,x2,y_ref,z_t,PID_action,PPO_action,combined_action,reward,error,steps_in_episode
0,3.208937,7.757649,3.0,0.000000,-42.010043,0.962120,-41.047923,-20.990612,-4.581551,1
1,2.891549,7.581551,3.0,-4.581551,-40.597827,0.091645,-40.506182,-19.315866,-4.394982,2
2,2.590266,7.394982,3.0,-8.976533,-39.050841,-1.000000,-40.050841,-17.625932,-4.198325,3
3,2.305111,7.198325,3.0,-13.174858,-37.320259,0.846032,-36.474227,-15.935776,-3.991964,4
4,2.036109,6.991964,3.0,-17.166822,-35.504283,-0.303175,-35.807458,-14.260318,-3.776284,5
...,...,...,...,...,...,...,...,...,...,...
399995,1.850783,1.947129,2.0,25.000000,0.840264,1.000000,1.840264,-0.003489,0.059065,1996
399996,1.977797,1.940935,2.0,25.000000,0.894773,-1.000000,-0.105227,-0.003219,0.056732,1997
399997,1.728624,1.943268,2.0,25.000000,0.874244,-1.000000,-0.125756,-0.005009,0.070772,1998
399998,1.495676,1.929228,2.0,25.000000,0.997792,-1.000000,-0.002208,-0.010036,0.100181,1999


# Separa as linhas por episódio

In [25]:
mask = rec_df["steps_in_episode"] == 1
start_episode_indexes = mask[mask].index.to_list()

first_episode_start = start_episode_indexes[0]
first_episode_end = start_episode_indexes[1] - 1 if len(start_episode_indexes) > 1 else len(rec_df) - 1

last_episode_start = start_episode_indexes[-1]
last_episode_end = len(rec_df) - 1

first_episode = rec_df.iloc[first_episode_start:first_episode_end+1]
last_episode = rec_df.iloc[last_episode_start:last_episode_end+1]

episodes_count = len(start_episode_indexes)

In [26]:
first_episode

,x1,x2,y_ref,z_t,PID_action,PPO_action,combined_action,reward,error,steps_in_episode
0,3.208937,7.757649,3.0,0.000000,-42.010043,0.962120,-41.047923,-20.990612,-4.581551,1
1,2.891549,7.581551,3.0,-4.581551,-40.597827,0.091645,-40.506182,-19.315866,-4.394982,2
2,2.590266,7.394982,3.0,-8.976533,-39.050841,-1.000000,-40.050841,-17.625932,-4.198325,3
3,2.305111,7.198325,3.0,-13.174858,-37.320259,0.846032,-36.474227,-15.935776,-3.991964,4
4,2.036109,6.991964,3.0,-17.166822,-35.504283,-0.303175,-35.807458,-14.260318,-3.776284,5
...,...,...,...,...,...,...,...,...,...,...
1995,1.177277,1.971653,2.0,24.566507,0.611449,1.000000,1.611449,-0.007206,0.084890,1996
1996,1.307325,1.915110,2.0,24.651398,1.111575,0.147241,1.258816,-0.016256,0.127500,1997
1997,1.356506,1.872500,2.0,24.778898,1.490366,-0.208768,1.281598,-0.026762,0.163591,1998
1998,1.406468,1.836409,2.0,24.942490,1.812879,1.000000,2.812879,-0.037469,0.193569,1999


In [27]:
last_episode

,x1,x2,y_ref,z_t,PID_action,PPO_action,combined_action,reward,error,steps_in_episode
398000,5.242792,7.276137,3.0,0.000000,-37.383286,-1.0,-38.383286,-17.672760,-4.203898,1
398001,4.837105,7.203898,3.0,-4.203898,-36.873705,1.0,-35.873705,-16.958139,-4.118026,2
398002,4.447430,7.118026,3.0,-8.321924,-36.241572,-1.0,-37.241572,-16.152130,-4.018971,3
398003,4.073781,7.018971,3.0,-12.340896,-35.490458,-1.0,-36.490458,-15.266031,-3.907177,4
398004,3.716172,6.907177,3.0,-16.248072,-34.623882,-1.0,-35.623882,-14.311687,-3.783079,5
...,...,...,...,...,...,...,...,...,...,...
399995,1.850783,1.947129,2.0,25.000000,0.840264,1.0,1.840264,-0.003489,0.059065,1996
399996,1.977797,1.940935,2.0,25.000000,0.894773,-1.0,-0.105227,-0.003219,0.056732,1997
399997,1.728624,1.943268,2.0,25.000000,0.874244,-1.0,-0.125756,-0.005009,0.070772,1998
399998,1.495676,1.929228,2.0,25.000000,0.997792,-1.0,-0.002208,-0.010036,0.100181,1999


### Plot Train over iteraçtion

In [28]:
def plot_progress(folder: str, save = False):
    # Plotting train/loss vs train/iteration
    plt.figure(figsize=(12, 6))
    plt.plot(progress_df['train/iteration'], progress_df['train/loss'], label='Train Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Train Loss vs. Iteration')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/loss_over_iteration.png")
    plt.show()

# plot_progress(latest_folder, save = True)

### Plote reward over steps

In [29]:
def plot_rewards(folder: str, save=False):
    # Plotting reward vs steps
    plt.figure(figsize=(12, 6))
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df['reward'], label='Reward')
    plt.xlabel('Steps')
    plt.ylabel('Reward')
    plt.title('Reward vs. Steps')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/reward_over_last_episode_steps.png")
    plt.show()

# plot_rewards(latest_folder, save=True)

### Plot Set point curve

In [30]:
def plot_tracking(folder: str, save: bool = False):
    # Plot y_ref vs steps_in_episode
    plt.figure(figsize=(12, 6))
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df[tracked_point], label="Altura da água no segundo tanque", color='red')
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df['y_ref'], '--', label='y_ref (Objetivo)', color='black')
    plt.xlabel('Steps')
    plt.ylabel('')
    plt.title('Altura da água no segundo tanque em cada passo no último episódio')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/tracking.png")
    plt.show()

# plot_tracking(latest_folder, save=True)

### Ação do PPO e PID durante o último episódio

In [31]:
def plot_action_per_steps(folder: str, save: bool = False, separeted: bool = True):

    if (separeted):
        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PID_action'], label='PID_action', color='blue')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação do controlador PID em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/PID_action.png")
        plt.show()

        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PPO_action'], label='PPO_action', color='red')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação do controlador PPO em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/PPO_action.png")
        plt.show()
    else:
        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PID_action'], label='PID_action', color='blue')
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PPO_action'], label='PPO_action', color='red')
        # plt.plot(last_episode_df['steps_in_episode'], last_episode_df['action'], label='Action', color='black')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/combined_action.png")
        plt.show()

# plot_action_per_steps(latest_folder, save=True)

# Função para exibir a grade de pastas

In [53]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def display_folders_grid(folder_names: list[str], selected_indices: int = 0) -> widgets.VBox:
    num_cols = 5
    previous_selected = [selected_indices]
    
    def on_button_click(b: widgets.Button):
        global selected_folder
        selected_folder = b.description

        global progress_df, rec_df, last_episode_df
        progress_df, rec_df, last_episode_df = load_dfs(selected_folder)

        previous_selected.append(buttons.index(b))
        if len(previous_selected) > 2:
            previous_selected.pop(0)
        
        # Atualiza cor dos botões
        buttons[previous_selected[0]].button_style = '' 
        buttons[previous_selected[1]].button_style = 'success'

        clear_output(wait=True)  # Limpar as saídas anteriores
        display(grid)            # Reexibir a grade de botões
        display(save_button)     # Reexibir o botão de salvar

        plot_tracking(selected_folder, save=False)
        plot_rewards(selected_folder, save=False)
        plot_action_per_steps(selected_folder, save=False)

    def on_save_click(b: widgets.Button):
        should_save = True  # Ativar o modo de salvar
        
        if 'selected_folder' in globals():
            plot_tracking(selected_folder, save=should_save)
            plot_rewards(selected_folder, save=should_save)
            plot_action_per_steps(selected_folder, save=should_save)
            print(f"Plots salvos para {selected_folder}!")
        else:
            print("Nenhuma pasta selecionada ainda!")

    # Criar os botões para as pastas
    buttons = []
    for i, folder in enumerate(folder_names):
        button = widgets.Button(description=folder, layout=widgets.Layout(width='auto', height='auto'))
        button.on_click(on_button_click)
        buttons.append(button)
    
    # Marcar o primeiro botão como ativo
    buttons[selected_indices].button_style = 'success'  
    
    # Criar o botão de salvar
    save_button = widgets.Button(description="Salvar Gráficos", button_style='info')
    save_button.on_click(on_save_click)

    # Criar a grade de botões
    grid = widgets.GridBox(
        children=buttons,
        layout=widgets.Layout(grid_template_columns=f'repeat({num_cols}, 1fr)')
    )

    display(save_button)
    display(grid)
    display(save_button)


In [54]:
# Uncomment to use widget 
display_folders_grid(folder_names, selected_index)

Button(button_style='info', description='Salvar Gráficos', style=ButtonStyle())

GridBox(children=(Button(description='11-02-2111', layout=Layout(height='auto', width='auto'), style=ButtonSty…

Button(button_style='info', description='Salvar Gráficos', style=ButtonStyle())